# Aplicamos transfer learning con un modelo que tienen todos los datos menos el de la especie a predecir

In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras 
import os
import json

import random as python_random
import time
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import MeanAbsolutePercentageError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.models import load_model

from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from keras import layers, models

np.random.seed(123)
python_random.seed(123)
tf.random.set_seed(1234)

from funcionesComunes import *

2024-10-29 12:29:40.324547: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-29 12:29:40.327305: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-29 12:29:40.336656: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-29 12:29:40.351613: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-29 12:29:40.356180: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 12:29:40.367870: I tensorflow/core/platform/cpu_feature_gu

In [41]:
for archivo in os.listdir("models/LSTMMerged_22_10_24"):

    # Leemos solo los archivos json para obtener los nombres
    if os.path.splitext(archivo)[1] == ".json":
        
        # Obtenemos el nombre de archivo
        nombreArchivo = archivo.split("_best_models.json")[0]

        print(f"PROCESANDO ARCHIVO: {nombreArchivo}")

        # Cargamos el modelo entrenado
        df = pd.read_csv(f'RCPMerged/{nombreArchivo}_merged.csv')
        df = df[~df["nametag"].str.startswith("INDI005")]

        # Codificamos, normalización y split de datos
        df = codification(df)
        print(df.shape)

        df, valorNormalizacion = individualNormalization(df)
        print(f"SE HA NORMALIZADO EL ARCHIVO: {archivo}")

        temp_df = df
        train_data, val_data, test_data = split_population_individuals(temp_df, train_pct=0.80, val_pct_in_train=0.20, details=False)
        train_data.shape, val_data.shape, test_data.shape

        # Obtenemos X e y para los datasets de train, val y test 
        WINDOWS_SIZE = 3
        X_train, y_train = df_to_X_y_ind_3(train_data, WINDOWS_SIZE)
        X_val, y_val = df_to_X_y_ind_3(val_data, WINDOWS_SIZE)
        X_test, y_test = df_to_X_y_ind_3(test_data, WINDOWS_SIZE)
        print(X_train.shape, X_test.shape, X_val.shape)

        # Cargamos el modelo global
        modelLSTM = tf.keras.models.load_model(f'models/LSTMMerged_22_10_24/{nombreArchivo}_model_1.keras')

        print(modelLSTM.summary())

        # Obtenemos el valor de batch size
        with open(f'models/LSTMMerged_22_10_24/{archivo}') as f:
            parameters = json.load(f)

        batch_size_LSTM = parameters[0]["batch_size"]

        historyLSTMTransfer = modelLSTM.fit(X_train, y_train, epochs=200, batch_size=batch_size_LSTM,
                            validation_data=(X_val, y_val), callbacks=[EarlyStopping(monitor='val_loss', patience=20)])
        
        modelLSTM.save(f"models/LSTMMerged_22_10_24_Trained/{nombreArchivo}.keras")

PROCESANDO ARCHIVO: Pinus wallichiana
(9365, 77)
SE HA NORMALIZADO EL ARCHIVO: Pinus wallichiana_best_models.json
(5508, 4, 43) (1957, 4, 43) (1500, 4, 43)


/home/workstation/Escritorio/PabloC/TransferLearning/.venv/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 40 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 4, 64)          │        27,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 4, 96)          │        61,824 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 4, 128)         │       115,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 4, 96)          │        86,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 128)            │       115,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 28)             │         3,612 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 28)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            29 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,229,741 (4.69 MB)

...

 Total params: 1,035,989 (3.95 MB)

 Trainable params: 345,329 (1.32 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 690,660 (2.63 MB)

None
Epoch 1/200
566/566 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - loss: 0.0151 - mape: 26.1009 - mse: 0.0151 - rmse: 0.1229 - val_loss: 0.0125 - val_mape: 20.1037 - val_mse: 0.0125 - val_rmse: 0.1119
Epoch 2/200
566/566 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 0.0145 - mape: 25.6281 - mse: 0.0145 - rmse: 0.1205 - val_loss: 0.0126 - val_mape: 20.4371 - val_mse: 0.0126 - val_rmse: 0.1122
Epoch 3/200
566/566 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 0.0143 - mape: 25.2914 - mse: 0.0143 - rmse: 0.1194 - val_loss: 0.0118 - val_mape: 20.7282 - val_mse: 0.0118 - val_rmse: 0.1085
Epoch 4/200
566/566 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 0.0143 - mape: 25.4909 - mse: 0.0143 - rmse: 0.1196 - val_loss: 0.0125 - val_mape: 20.4723 - val_mse: 0.0125 - val_rmse: 0.1117
Epoch 5/200
566/566 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 0.0144 - mape: 25.5541 - mse: 0.0144 - rmse: 0.1199 - val_loss: 0.0119 - val_mape: 20.6555 - val_mse: 0.0119 - val_rmse: 0.1092
Epoch 6/200
566/566 ━━━━━━━━━━━━━━━━━━━━ 6s 

In [43]:
# Creamos un df que contiene los resultados de los modelos freeze de cada especie
df_Freeze = pd.DataFrame(columns = ["File", "Time", "TrainMSE", "TrainRMSE", "TrainR2", "TrainMAPE", 
                                    "ValidationMSE", "ValidationRMSE", "ValidationR2", "ValidationMAPE",
                                    "TestMSE", "TestRMSE", "TestR2", "TestMAPE"])

for archivo in os.listdir("models/LSTMMerged_22_10_24"):

    # Leemos solo los archivos json para obtener los nombres
    if os.path.splitext(archivo)[1] == ".json":
        
        # Obtenemos el nombre de archivo
        nombreArchivo = archivo.split("_best_models.json")[0]

        print(f"PROCESANDO ARCHIVO: {nombreArchivo}")

        # Cargamos el modelo entrenado
        df = pd.read_csv(f'RCPMerged/{nombreArchivo}_merged.csv')
        df = df[~df["nametag"].str.startswith("INDI005")]

        # Codificamos, normalización y split de datos
        df = codification(df)
        print(df.shape)

        df, valorNormalizacion = individualNormalization(df)
        print(f"SE HA NORMALIZADO EL ARCHIVO: {archivo}")

        temp_df = df
        train_data, val_data, test_data = split_population_individuals(temp_df, train_pct=0.80, val_pct_in_train=0.20, details=False)
        train_data.shape, val_data.shape, test_data.shape

        # Obtenemos X e y para los datasets de train, val y test 
        WINDOWS_SIZE = 3
        X_train, y_train = df_to_X_y_ind_3(train_data, WINDOWS_SIZE)
        X_val, y_val = df_to_X_y_ind_3(val_data, WINDOWS_SIZE)
        X_test, y_test = df_to_X_y_ind_3(test_data, WINDOWS_SIZE)
        print(X_train.shape, X_test.shape, X_val.shape)

        # Cargamos el modelo global
        modelLSTM = tf.keras.models.load_model(f'models/LSTMMerged_22_10_24_Trained/{nombreArchivo}.keras')
        modelLSTMFreeze = modelLSTM

        # Obtener el optimizador del modelo
        optimizer = modelLSTMFreeze.optimizer

        # Obtenemos el valor de batch size
        with open(f'models/LSTMMerged_22_10_24/{archivo}') as f:
            parameters = json.load(f)

        batch_size_LSTM = parameters[0]["batch_size"]

        # Indicamos el numero de layers a entrenar
        NUM_TRAINABLE = 1

        numLSTM_layers = sum(1 for layer in modelLSTMFreeze.layers if "lstm" in layer.name)

        numFreezeLayers = numLSTM_layers - NUM_TRAINABLE # Congelamos todas menos la útlima capa

        numberLayersFreezed = 0

        # Congelar las primeras 'numFreezeLayers' capas LSTM
        for i, layer in enumerate(modelLSTMFreeze.layers):

            if "lstm" in layer.name and numFreezeLayers>numberLayersFreezed:
                numberLayersFreezed += 1
                layer.trainable = False

            print(layer, layer.trainable)

        # Compilamos el modelo con los nuevos datos
        modelLSTMFreeze.compile(optimizer=optimizer, loss=MeanSquaredError(), metrics=[
                                    MeanSquaredError(name='mse'),
                                    RootMeanSquaredError(name='rmse'),
                                    MeanAbsolutePercentageError(name='mape')
                                ])

        # Comienza a medir el tiempo de entrenamiento
        start_time = time.time()

        historyLSTMTransfer = modelLSTMFreeze.fit(X_train, y_train, epochs=200, batch_size=batch_size_LSTM,
                                validation_data=(X_val, y_val), callbacks=[EarlyStopping(monitor='val_loss', patience=10)])
        
        # Finaliza la medición del tiempo de entrenamiento
        end_time = time.time()
        print(f"[MODELO CONGELADO]: {end_time-start_time}")

        # Realizar predicciones y calcular métricas para el conjunto de entrenamiento
        predictions_train = predictionForIndividuals(X_train, y_train, modelLSTMFreeze, batch_size_LSTM)
        predictions_train["PredictionsDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_train["ActualDenormalize"] = predictions_train.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        train_mse = mean_squared_error(predictions_train["ActualDenormalize"],predictions_train["PredictionsDenormalize"])
        train_rmse = np.sqrt(train_mse)
        train_mape = (np.sum(np.abs(predictions_train["PredictionsDenormalize"] - predictions_train["ActualDenormalize"])) / np.sum(np.abs(predictions_train["ActualDenormalize"]))) * 100
        train_r2 = r2_score(predictions_train["ActualDenormalize"], predictions_train["PredictionsDenormalize"])

        # Realizar predicciones y calcular métricas para el conjunto de validación
        predictions_val = predictionForIndividuals(X_val, y_val, modelLSTMFreeze, batch_size_LSTM)
        predictions_val["PredictionsDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_val["ActualDenormalize"] = predictions_val.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        val_mse = mean_squared_error(predictions_val["ActualDenormalize"],predictions_val["PredictionsDenormalize"])
        val_rmse = np.sqrt(val_mse)
        val_mape = (np.sum(np.abs(predictions_val["PredictionsDenormalize"] - predictions_val["ActualDenormalize"])) / np.sum(np.abs(predictions_val["ActualDenormalize"]))) * 100
        val_r2 = r2_score(predictions_val["ActualDenormalize"], predictions_val["PredictionsDenormalize"])

        # Realizar predicciones y calcular métricas para el conjunto de prueba
        predictions_test = predictionForIndividuals(X_test, y_test, modelLSTMFreeze, batch_size_LSTM)
        predictions_test["PredictionsDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Predictions"), axis=1)
        predictions_test["ActualDenormalize"] = predictions_test.apply(lambda row: desnormalizacionBAI(row, valorNormalizacion, "Actuals"), axis=1)

        test_mse = mean_squared_error(predictions_test["ActualDenormalize"],predictions_test["PredictionsDenormalize"])
        test_rmse = np.sqrt(test_mse)
        test_mape = (np.sum(np.abs(predictions_test["PredictionsDenormalize"] - predictions_test["ActualDenormalize"])) / np.sum(np.abs(predictions_test["ActualDenormalize"]))) * 100
        test_r2 = r2_score(predictions_test["ActualDenormalize"], predictions_test["PredictionsDenormalize"])

        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Train): {train_mse}, {train_rmse}, {train_r2}, {train_mape}")
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Val): {val_mse}, {val_rmse}, {val_r2}, {val_mape}")
        print(f"RESULTADOS DE MSE, RMSE, R2, MAPE (Test): {test_mse}, {test_rmse}, {test_r2}, {test_mape}")

        # Guardamos los datos calculados
        df_Freeze.loc[len(df_Freeze)] = [nombreArchivo, end_time-start_time,train_mse, train_rmse, train_r2, train_mape, val_mse, val_rmse, val_r2, val_mape, test_mse, test_rmse, test_r2, test_mape]

df_Freeze.to_csv(f'resultsTransferOption2/resultados_Option2.csv', index=False)

PROCESANDO ARCHIVO: Pinus wallichiana
(9365, 77)
SE HA NORMALIZADO EL ARCHIVO: Pinus wallichiana_best_models.json
(5508, 4, 43) (1957, 4, 43) (1500, 4, 43)
<LSTM name=lstm, built=True> False
<LSTM name=lstm_1, built=True> False
<LSTM name=lstm_2, built=True> False
<Dropout name=dropout, built=True> True
<LSTM name=lstm_3, built=True> False
<LSTM name=lstm_4, built=True> True
<Dense name=dense, built=True> True
<Dropout name=dropout_1, built=True> True
<Dense name=dense_1, built=True> True
Epoch 1/200
345/345 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 0.0108 - mape: 20.3181 - mse: 0.0108 - rmse: 0.1038 - val_loss: 0.0109 - val_mape: 19.9513 - val_mse: 0.0109 - val_rmse: 0.1042
Epoch 2/200
345/345 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 0.0104 - mape: 19.7378 - mse: 0.0104 - rmse: 0.1018 - val_loss: 0.0108 - val_mape: 20.0018 - val_mse: 0.0109 - val_rmse: 0.1040
Epoch 3/200
345/345 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - loss: 0.0104 - mape: 19.7699 - mse: 0.0104 - rmse: 0.1021 - val_loss: 0.0

/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 6.121465170418535, 2.474159487668193, 0.9091929493359101, 14.227970213192243
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 5.848367371314177, 2.418339796495558, 0.9021739707723718, 15.69187719907856
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 8.068032541857075, 2.840428232125761, 0.8086495324397857, 18.949196142172305
PROCESANDO ARCHIVO: Pinus gerardiana
(10101, 77)
SE HA NORMALIZADO EL ARCHIVO: Pinus gerardiana_best_models.json
(5767, 4, 43) (2181, 4, 43) (1745, 4, 43)
<LSTM name=lstm, built=True> False
<LSTM name=lstm_1, built=True> False
<LSTM name=lstm_2, built=True> False
<Dropout name=dropout, built=True> True
<LSTM name=lstm_3, built=True> False
<LSTM name=lstm_4, built=True> True
<Dense name=dense, built=True> True
<Dropout name=dropout_1, built=True> True
<Dense name=dense_1, built=True> True
Epoch 1/200
361/361 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.0150 - mape: 30.9217 - mse: 0.0150 - rmse: 0.1226 - val_loss: 0.0194 - val_mape: 38.3674

/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 6.051764715796212, 2.46003347859256, 0.844095497177946, 19.31026214769602
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 10.556041233049728, 3.2490061915991677, 0.7332310743298152, 23.27716508352816
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 8.518140532158116, 2.918585364891374, 0.7879764608064388, 22.22331630156465
PROCESANDO ARCHIVO: Betula utilis
(3473, 77)
SE HA NORMALIZADO EL ARCHIVO: Betula utilis_best_models.json
(2059, 4, 43) (665, 4, 43) (593, 4, 43)
<LSTM name=lstm_3, built=True> False
<LSTM name=lstm_4, built=True> False
<LSTM name=lstm_5, built=True> False
<Dropout name=dropout_2, built=True> True
<LSTM name=lstm_6, built=True> False
<LSTM name=lstm_7, built=True> True
<Dense name=dense_2, built=True> True
<Dropout name=dropout_3, built=True> True
<Dense name=dense_3, built=True> True
Epoch 1/200
129/129 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0108 - mape: 38.8665 - mse: 0.0108 - rmse: 0.1039 - val_loss: 0.0111 - val_mape: 24.0085 - v

...

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 2.610944396957065, 1.6158416992258446, 0.9418326603922483, 16.59673253155935
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 2.036629215490828, 1.427105187255245, 0.8815644927382139, 19.17263492296867
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 0.9520484685375733, 0.9757297107998574, 0.8248496731629177, 19.57369523421814
PROCESANDO ARCHIVO: Populus ciliata
(1182, 77)
SE HA NORMALIZADO EL ARCHIVO: Populus ciliata_best_models.json
(705, 4, 43) (156, 4, 43) (265, 4, 43)
<LSTM name=lstm_4, built=True> False
<LSTM name=lstm_5, built=True> False
<Dropout name=dropout_2, built=True> True
<LSTM name=lstm_6, built=True> False
<LSTM name=lstm_7, built=True> True
<Dense name=dense_2, built=True> True
<Dropout name=dropout_3, built=True> True
<Dense name=dense_3, built=True> True
Epoch 1/200
45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - loss: 0.0127 - mape: 31.1091 - mse: 0.0127 - rmse: 0.1124 - val_loss: 0.0202 - val_mape: 40.6397 - val_mse: 0.0202 - val_rmse: 0.1423

/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 82.66292570727003, 9.091915403657802, 0.7723993488441996, 22.919676872657604
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 102.85194358129758, 10.141594725746911, 0.5617689964123171, 30.312853700301613
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 55.45381489524232, 7.446731826462016, 0.7229108353410518, 27.483927656319207
PROCESANDO ARCHIVO: Abies pindrow
(16018, 77)
SE HA NORMALIZADO EL ARCHIVO: Abies pindrow_best_models.json
(9042, 4, 43) (3441, 4, 43) (2815, 4, 43)
<LSTM name=lstm, built=True> False
<LSTM name=lstm_1, built=True> False
<LSTM name=lstm_2, built=True> False
<Dropout name=dropout, built=True> True
<LSTM name=lstm_3, built=True> False
<LSTM name=lstm_4, built=True> False
<LSTM name=lstm_5, built=True> True
<Dense name=dense, built=True> True
<Dropout name=dropout_1, built=True> True
<Dense name=dense_1, built=True> True
Epoch 1/200
566/566 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - loss: 0.0123 - mape: 23.0604 - mse: 0.0123 - rmse: 0.1107 - va

/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictionsDF = pd.concat([predictionsDF, tempDF], ignore_index=True)
/home/workstation/Escritorio/PabloC/TransferLearning/funcionesComunes.py:247: FutureWarning: The behavior of Dat

RESULTADOS DE MSE, RMSE, R2, MAPE (Train): 43.852303891892724, 6.622107209332444, 0.811771901779304, 20.04436864533362
RESULTADOS DE MSE, RMSE, R2, MAPE (Val): 35.49411745545523, 5.957693971282448, 0.7975242259421406, 18.86939277300311
RESULTADOS DE MSE, RMSE, R2, MAPE (Test): 47.760894465168526, 6.910925731417501, 0.6623439720754176, 20.120572543870928
